Testing out warping of images here

In [15]:
import glob
import os.path
import pickle

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from moviepy.editor import VideoFileClip

import cv2
%matplotlib osx

def get_calibration_data():
    pickle_file = 'cal_data/cal_data.p'
    print("Hello")
    if os.path.exists(pickle_file):
        print("loading pickle file for calibration", pickle_file)
        x = pickle.load(open(pickle_file, "rb"))
        mtx = x['mtx']
        dist = x['dist']
    else:
        x, mtx, dist, x, x = calibrate_camera()
        pickle.dump({'mtx': mtx, 'dist': dist}, open(pickle_file, 'wb'))
    return mtx, dist

# More stuff

In [13]:
def test_calibration(image, mtx, dist, folderPath='camera_cal'):
    img = cv2.imread(folderPath + '/' + image)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    #cv2.imshow('undist', undist)
    cv2.imwrite('output_images/calibrated_' + image, undist)
    
def undistort_frame(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)


In [16]:
def test_stuff():
    img = cv2.imread("test_images/straight_lines2.jpg")
    mtx, dist = get_calibration_data()
    undistorted = undistort_frame()
    test_calibration('signs_vehicles_xygrad.png', mtx, dist, 'test_images')

test_stuff()

Hello
loading pickle file for calibration cal_data/cal_data.p


In [10]:
cv2.destroyAllWindows()